In [1]:
import os
import librosa
import soundfile as sf
from librosa.core import load, stft, istft, magphase
from config import *
from concurrent.futures import ThreadPoolExecutor   
from time import time
import asyncio
import numpy as np
from multiprocessing import cpu_count

In [6]:
dirs_dev = list(os.walk('DSD100_extracted/Dev/'))[0][1]
dirs_test = list(os.walk('DSD100_extracted/Test/'))[0][1]

def downsample(input_path, output_path):
    wav, _ = librosa.load(input_path, sr=SAMPLE_RATE)
    sf.write(output_path, wav, SAMPLE_RATE)
    print(f"Saving {output_path}")


In [7]:
if __name__ == '__main__':
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs_dev)):
            target_dir = 'DSD100_resized/Dev/{}/'.format(dirs_dev[i])
            os.makedirs(target_dir, exist_ok=True)                    
            pool.submit(downsample, f'DSD100_extracted/Dev/{dirs_dev[i]}/mixture.wav', target_dir + 'mixture.wav')
            pool.submit(downsample, f'DSD100_extracted/Dev/{dirs_dev[i]}/vocals.wav', target_dir + 'vocals.wav')
            pool.submit(downsample, f'DSD100_extracted/Dev/{dirs_dev[i]}/accompaniment.wav', target_dir + 'accompaniment.wav')
        for i in range(len(dirs_test)):
            target_dir = 'DSD100_resized/Test/{}/'.format(dirs_test[i])
            os.makedirs(target_dir, exist_ok=True)                    
            pool.submit(downsample, f'DSD100_extracted/Test/{dirs_test[i]}/mixture.wav', target_dir + 'mixture.wav')
            pool.submit(downsample, f'DSD100_extracted/Test/{dirs_test[i]}/vocals.wav', target_dir + 'vocals.wav')
            pool.submit(downsample, f'DSD100_extracted/Test/{dirs_test[i]}/accompaniment.wav', target_dir + 'accompaniment.wav')

Saving DSD100_resized/Test/018 - Hollow Ground - Ill Fate/mixture.wav
Saving DSD100_resized/Dev/054 - Actions - South Of The Water/vocals.wav
Saving DSD100_resized/Dev/053 - Actions - Devil's Words/accompaniment.wav
Saving DSD100_resized/Dev/053 - Actions - Devil's Words/vocals.wav
Saving DSD100_resized/Dev/053 - Actions - Devil's Words/mixture.wav
Saving DSD100_resized/Dev/052 - ANiMAL - Easy Tiger/accompaniment.wav
Saving DSD100_resized/Dev/052 - ANiMAL - Easy Tiger/mixture.wav
Saving DSD100_resized/Dev/051 - AM Contra - Heart Peripheral/accompaniment.wav


In [8]:
def load_as_mag(file):
    wav, _ = librosa.load(file, sr=None)
    spectrogram = stft(wav, n_fft=WINDOW_SIZE, hop_length=HOP_LENGTH)
    mag, _ = magphase(spectrogram)
    return mag.astype(np.float32)

def save_to_npz(base, sub, sample):
    nps = {}
    mix = load_as_mag(f'{base}/{sample}/mixture.wav')
    vocal = load_as_mag(f'{base}/{sample}/vocals.wav')
    acc = load_as_mag(f'{base}/{sample}/accompaniment.wav')
        
    mix_max = mix.max()
    mix_norm = mix / mix_max
    vocal_norm = vocal / mix_max
    acc_norm = acc / mix_max
    
    if not os.path.exists(f'DSD100_Npz/{sub}'):
        os.makedirs(f'DSD100_Npz/{sub}')
    
    print(f"Saving {sample}")
    np.savez_compressed(f'DSD100_Npz/{sub}/{sample}.npz', mix=mix_norm, vocal=vocal_norm, acc=acc_norm)
    
if __name__ == '__main__':
    if not os.path.exists('DSD100_Npz'):
        os.makedirs('DSD100_Npz')
    dirs_dev = list(os.walk('DSD100_extracted/Dev/'))[0][1]
    dirs_test = list(os.walk('DSD100_extracted/Test/'))[0][1]
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs_dev)):
            pool.submit(save_to_npz, 'DSD100_resized/Dev', 'Dev', dirs_dev[i])
        for i in range(len(dirs_test)):
            pool.submit(save_to_npz, 'DSD100_resized/Test', 'Test', dirs_test[i])

Saving 053 - Actions - Devil's Words
